In [1]:
# Imports

from argparse import Namespace
import pickle
from pathlib import Path
import numpy as np

from probo import NelderMeadAcqOptimizer, SimpleBo
from branin import branin, get_branin_domain_nd
from penn_sklearn import SklearnPenn

In [2]:
# Set seed

seed = 11
np.random.seed(seed)

In [3]:
# BO Setup

# define function
n_dim = 40
f = lambda x: np.sum([branin(x[2 * i : 2 * i + 2]) for i in range(n_dim // 2)])

# define model
model = SklearnPenn()

# define acqfunction
acqfunction = {'acq_str': 'ts', 'n_gen': 500}

# define acqoptimizer
domain = get_branin_domain_nd(n_dim)
acqoptimizer = NelderMeadAcqOptimizer({'rand_every': 10, 'max_iter': 200, 'jitter': True}, domain)

# define initial dataset
n_init = 80
data = Namespace()
data.x = domain.unif_rand_sample(n_init)
data.y = [f(x) for x in data.x]

# define BO
n_iter = 3
bo = SimpleBo(f, model, acqfunction, acqoptimizer, data=data, params={'n_iter': n_iter}, seed=seed)

*[INFO] Model=SklearnPenn with params=Namespace(alpha=0.01, hls=(20, 30, 40), max_iter=500, n_ensemble=5, trans_x=False)
*[INFO] RealDomain with params=Namespace(dom_str='real', min_max=[(-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15)])
*[INFO] NelderMeadAcqOptimizer with params=Namespace(domain_params=Namespace(dom_str='real', min_max=[(-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10), (0, 15), (-5, 10

In [4]:
# Run BO

results = bo.run()

*[KEY] i: iteration, x: design, y: objective, min_y: minimum objective so far (* indicates a new min_y)
i: 0,    x: [1.88786861379254, 5.646327594.. 	y: 1042.3721,	min_y: 701.7171 
i: 1,    x: [-5.0, 13.72504093764692, 3.25.. 	y: 680.2449,	min_y: 680.2449 *
i: 2,    x: [-5.0, 14.417920016883839, 3.7.. 	y: 750.5265,	min_y: 680.2449 
Minimum y = 680.244940152023
Minimizer x = [-5.0, 13.72504093764692, 3.2501219939817894, 3.8398430363348504, -2.276275359980508, 11.06459751547162, -0.3266933524995721, 13.01996086916991, 3.3578572458316804, 4.299315524861031, -1.9754052820519106, 6.446597619870904, 9.439424973361355, 1.282277450465883, 3.4513778119361027, 4.754938330930675, -4.10437485207705, 3.5254148504242315, 9.341950919377444, 3.254054624374378, 4.190876931670375, 12.90557474926445, 1.3429644201622026, 7.41375242313291, 4.500619421664984, 8.244378157727777, 8.304308697356399, 6.034489275111271, 1.031125524526684, 7.051361166933784, -3.329233391290358, 9.46572297148492, -1.42395165034419

In [5]:
# Save results

save_dir = Path(f'results/branin{n_dim}_{seed}')
save_dir.mkdir(parents=True, exist_ok=True)

with open(save_dir / "results.pkl", "wb") as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)